In [1]:
import json

In [2]:
def rename_metric(metric):
    if metric == 'F-measure':
        return 'f1'
    if metric == 'Accuracy':
        return 'acc'
    if metric == 'Runtime':
        return 'time'
    raise ValueError('Unknown metric')

In [3]:
def maybe_add_zero(mean, std):
    if len(mean.split('.')[-1]) == 1:
        mean += '0'
    if len(std.split('.')[-1]) == 1:
        std += '0'
    return mean, std

In [4]:
def get_results(metric, kb_emb_model):
    Res = {metric: {'Carcinogenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{m\\text{=32}}$': '', 'NCES$_{m\\text{=64}}$': '',\
                                       'NCES$_{m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64}}$': '', 'NCES$_{m\\text{=32+}m\\text{=128}}$': '',\
                                       'NCES$_{m\\text{=64+}m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64+}m\\text{=128}}$': ''},
                   'Mutagenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{m\\text{=32}}$': '', 'NCES$_{m\\text{=64}}$': '',\
                                       'NCES$_{m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64}}$': '', 'NCES$_{m\\text{=32+}m\\text{=128}}$': '',\
                                       'NCES$_{m\\text{=64+}m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64+}m\\text{=128}}$': ''},
                    'Semantic Bible': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{m\\text{=32}}$': '', 'NCES$_{m\\text{=64}}$': '',\
                                       'NCES$_{m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64}}$': '', 'NCES$_{m\\text{=32+}m\\text{=128}}$': '',\
                                       'NCES$_{m\\text{=64+}m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64+}m\\text{=128}}$': ''},
                    'Vicodi': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{m\\text{=32}}$': '', 'NCES$_{m\\text{=64}}$': '',\
                                       'NCES$_{m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64}}$': '', 'NCES$_{m\\text{=32+}m\\text{=128}}$': '',\
                                       'NCES$_{m\\text{=64+}m\\text{=128}}$': '', 'NCES$_{m\\text{=32+}m\\text{=64+}m\\text{=128}}$': ''}
                   }}
    for kb in ['carcinogenesis', 'mutagenesis', 'semantic_bible', 'vicodi']:
        for model in ['celoe', 'eltl', 'ecii', 'evolearner', f'NCES_{kb_emb_model}', f'NCES_{kb_emb_model}_Ensemble']:
            if model not in [f'NCES_{kb_emb_model}', f'NCES_{kb_emb_model}_Ensemble']:
                rename_model = model.upper() if model != 'evolearner' else 'EvoLearner'
                with open(f'datasets/{kb}/Results/{rename_model}_avg_results.json') as file:
                    results = json.load(file)
                mean = str(round(results[metric]['mean'],2)); std = str(round(results[metric]['std'],2))
                mean, std = maybe_add_zero(mean, std)
                Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
            elif model == f'NCES_{kb_emb_model}':
                path = f'datasets/{kb}/Results/{model}.json'
                if kb_emb_model == "DistMult":
                    path = f'datasets/{kb}/Results/{model}_{kb_emb_model}.json'
                with open(path) as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results: 
                    inducing_points = m.split("I")[-1]
                    rename_model = 'NCES$_{m\\text{='+inducing_points+'}}$'
                    mean = str(round(results[m][new_metric]['mean'][0],2))
                    std = str(round(results[m][new_metric]['std'][0],2))
                    mean, std = maybe_add_zero(mean, std)
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
            elif model == f'NCES_{kb_emb_model}_Ensemble':
                path = f'datasets/{kb}/Results/{model}.json'
                if kb_emb_model == "DistMult":
                    path = f'datasets/{kb}/Results/NCES_DistMult_Ensemble.json'
                with open(path) as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results:
                    inducing_points = [name.split("I")[-1] for name in m.split("+")]
                    new_name = ''
                    for i,ind_pt in enumerate(inducing_points):
                        if i != len(inducing_points)-1:
                            new_name += 'm\\text{='+ind_pt+'+}'
                        else:
                            new_name += 'm\\text{='+ind_pt+'}'
                    rename_model = 'NCES$_{'+new_name+'}$'
                    mean = str(round(results[m][new_metric]['mean'][0],2)); std = str(round(results[m][new_metric]['std'][0],2))
                    mean, std = maybe_add_zero(mean, std)
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] = mean+'$\pm$'+std
    return Res

In [5]:
Res1 = get_results('F-measure', 'ConEx')

In [6]:
Res1

{'F-measure': {'Carcinogenesis': {'CELOE': '29.24$\\pm$39.22',
   'ELTL': '14.46$\\pm$28.48',
   'ECII': '18.91$\\pm$31.46',
   'EvoLearner': '89.34$\\pm$15.80',
   'NCES$_{m\\text{=32}}$': '83.56$\\pm$33.11',
   'NCES$_{m\\text{=64}}$': '83.92$\\pm$33.16',
   'NCES$_{m\\text{=128}}$': '86.06$\\pm$32.63',
   'NCES$_{m\\text{=32+}m\\text{=64}}$': '92.13$\\pm$24.61',
   'NCES$_{m\\text{=32+}m\\text{=128}}$': '91.01$\\pm$26.80',
   'NCES$_{m\\text{=64+}m\\text{=128}}$': '92.57$\\pm$24.08',
   'NCES$_{m\\text{=32+}m\\text{=64+}m\\text{=128}}$': '91.29$\\pm$24.96'},
  'Mutagenesis': {'CELOE': '74.46$\\pm$37.59',
   'ELTL': '36.33$\\pm$34.98',
   'ECII': '34.33$\\pm$31.53',
   'EvoLearner': '95.37$\\pm$8.02',
   'NCES$_{m\\text{=32}}$': '76.79$\\pm$38.61',
   'NCES$_{m\\text{=64}}$': '78.25$\\pm$37.18',
   'NCES$_{m\\text{=128}}$': '73.21$\\pm$38.31',
   'NCES$_{m\\text{=32+}m\\text{=64}}$': '83.09$\\pm$34.04',
   'NCES$_{m\\text{=32+}m\\text{=128}}$': '86.33$\\pm$31.54',
   'NCES$_{m\\text{

In [5]:
def write_results(metric, kb_emb_model):
    data = get_results(metric, kb_emb_model)
    with open(f'{kb_emb_model}_{metric}.txt', 'w') as file:
        for key in data:
            file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
            file.write("\t\t\\toprule\n")
            file.write("\t\t& \\multicolumn{4}{c}{"+"$"+metric+"(\%)$}\\\\\n")
            file.write("\t\t&"+" & ".join(["\\textbf{"+kb+"}" for kb in data[key]])+"\\\\\n")
            file.write("\t\t\\midrule\n")
            for model in data[key]['Carcinogenesis']:
                file.write("\t\t"+model+" & "+" & ".join([data[key][kb][model] for kb in data[key]])+"\\\\\n")

In [16]:
write_results('F-measure', 'ConEx')

In [17]:
write_results('Accuracy', 'ConEx')

In [18]:
write_results('Runtime', 'ConEx')

In [9]:
write_results('F-measure', 'Distmult')

In [10]:
write_results('Accuracy', 'Distmult')

In [11]:
write_results('Runtime', 'Distmult')